# Minimal 3-node example of PyPSA power flow

Available as a Jupyter notebook at <http://www.pypsa.org/examples/minimal_example_pf.ipynb>.

In [1]:

# make the code as Python 3 compatible as possible
from __future__ import print_function, division
import pypsa

import numpy as np


In [2]:
network = pypsa.Network()

In [3]:
#add three buses
n_buses = 3

for i in range(n_buses):
    network.add("Bus","My bus {}".format(i),
                v_nom=20.)

In [4]:
print(network.buses)

attribute  v_nom    x    y carrier  v_mag_pu_set  v_mag_pu_min  v_mag_pu_max  \
My bus 0    20.0  0.0  0.0      AC           1.0           0.0           inf   
My bus 1    20.0  0.0  0.0      AC           1.0           0.0           inf   
My bus 2    20.0  0.0  0.0      AC           1.0           0.0           inf   

attribute control sub_network  
My bus 0       PQ              
My bus 1       PQ              
My bus 2       PQ              


In [5]:
#add three lines in a ring
for i in range(n_buses):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%n_buses),
                x=0.1,
                r=0.01)

In [6]:
print(network.lines)

attribute      bus0      bus1 type    x     r    g    b  s_nom  \
My line 0  My bus 0  My bus 1       0.1  0.01  0.0  0.0    0.0   
My line 1  My bus 1  My bus 2       0.1  0.01  0.0  0.0    0.0   
My line 2  My bus 2  My bus 0       0.1  0.01  0.0  0.0    0.0   

attribute s_nom_extendable  s_nom_min    ...      length  terrain_factor  \
My line 0            False        0.0    ...         0.0             1.0   
My line 1            False        0.0    ...         0.0             1.0   
My line 2            False        0.0    ...         0.0             1.0   

attribute  v_ang_min  v_ang_max  sub_network  x_pu r_pu  g_pu  b_pu  s_nom_opt  
My line 0       -inf        inf                0.0  0.0   0.0   0.0        0.0  
My line 1       -inf        inf                0.0  0.0   0.0   0.0        0.0  
My line 2       -inf        inf                0.0  0.0   0.0   0.0        0.0  

[3 rows x 22 columns]


In [7]:
#add a generator at bus 0
network.add("Generator","My gen",
            bus="My bus 0",
            p_set=100,
            control="PQ")

In [8]:
print(network.generators)

attribute       bus control  p_nom p_nom_extendable  p_nom_min  p_nom_max  \
My gen     My bus 0      PQ    0.0            False        0.0        inf   

attribute  p_min_pu  p_max_pu  p_set  q_set  sign carrier  marginal_cost  \
My gen          0.0       1.0  100.0    0.0   1.0                    0.0   

attribute  capital_cost  efficiency  p_nom_opt  
My gen              0.0         1.0        0.0  


In [9]:
print(network.generators.p_set)

My gen    100.0
Name: p_set, dtype: float64


In [10]:
#add a load at bus 1
network.add("Load","My load",
            bus="My bus 1",
            p_set=100)

In [11]:
print(network.loads)

attribute       bus  p_set  q_set  sign
My load    My bus 1  100.0    0.0  -1.0


In [12]:
print(network.loads.p_set)

My load    100.0
Name: p_set, dtype: float64


In [19]:
network.loads.q_set = 100.

In [20]:
#Do a Newton-Raphson power flow
network.pf()

INFO:pypsa.pf:Slack bus for sub-network 0 is My bus 0
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index([u'now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.032982 seconds


In [21]:
print(network.lines_t.p0)

     My line 0  My line 1  My line 2
now  66.897487 -33.333333 -33.391038


In [22]:
print(network.buses_t.v_ang*180/np.pi)

     My bus 0  My bus 1  My bus 2
now       0.0 -0.875939 -0.433813


In [23]:
print(network.buses_t.v_mag_pu)

     My bus 0  My bus 1  My bus 2
now       1.0  0.981199   0.99057
